- use google places api given the coordinates of MRT stations
- retrieve cafe data given 500m radius (1km diameter)

In [5]:
import pandas as pd
import json
import os
import geopandas as gpd
import requests
import time
import googlemaps

# Prep API fields

In [4]:
# import api key
api_key = os.getenv('GOOGLE_MAPS_API_KEY')

# set up googlemaps client
gmaps = googlemaps.Client(key=api_key)

In [ ]:
# import MRT station locations
mrt_stations = pd.read_csv('data/transit_stations.csv')

# merge lat and lng into tuple in a new column
mrt_stations['location'] = list(zip(mrt_stations['station_lat'], mrt_stations['station_lng']))

# remove bus stops
mrt_stations = mrt_stations[mrt_stations['station_type'] == 'MRT']

# format into list of tuple coordinatesto be passed into API call
mrt_locations = [mrt_stations['location'][i] for i in range(len(mrt_stations))]

In [73]:
mrt_stations

,station_name,station_line,station_type,station_lat,station_lng,location
0,Waterfront,Expo,MRT,49.286075,-123.111738,"(49.2860754844223, -123.111738155049)"
1,VCC - Clark,Expo,MRT,49.265783,-123.078962,"(49.2657831984991, -123.078962252431)"
2,Olympic Village,Canada,MRT,49.266340,-123.115197,"(49.2663402324943, -123.115196678993)"
3,Broadway - City Hall,Canada,MRT,49.262773,-123.114782,"(49.2627734678834, -123.114782181391)"
4,King Edward,Canada,MRT,49.249123,-123.115336,"(49.2491228783803, -123.115336104847)"
5,Burrard,Expo,MRT,49.285860,-123.119972,"(49.2858601491736, -123.119972336442)"
6,Marine Drive,Canada,MRT,49.209548,-123.117071,"(49.209547873924, -123.117070776267)"
7,Granville,Expo,MRT,49.283638,-123.116404,"(49.2836376868005, -123.116404027195)"
8,Stadium - Chinatown,Expo,MRT,49.279442,-123.109565,"(49.2794416918776, -123.109564796303)"
9,Main Street - Science World,Expo,MRT,49.273178,-123.100607,"(49.2731779118796, -123.100606907688)"


In [76]:
# convert mrt stations into geodataframe for use in tableau
mrt_gdf = gpd.GeoDataFrame(mrt_stations, geometry=gpd.points_from_xy(mrt_stations.station_lng, mrt_stations.station_lat))

mrt_gdf.drop(columns=['location', 'station_lat', 'station_lng', 'station_type'], inplace=True)

# export as geojson
mrt_gdf.to_file('data/mrt_stations.geojson', driver='GeoJSON')

In [24]:
# Define a function to retrieve all venues for a given location
def get_all_venues(location, radius, venue_type):
    initial_result = gmaps.places_nearby(location=location, radius=radius, type=venue_type)
    all_results = initial_result.get('results', [])

    while 'next_page_token' in initial_result:
        time.sleep(3)  # Necessary sleep before using the next page token
        initial_result = gmaps.places_nearby(page_token=initial_result['next_page_token'])
        all_results.extend(initial_result.get('results', []))

    return all_results

# Initialize an empty list to store all venues
cafe_list = []

# Loop through each coordinate
for lat, lng in mrt_locations:
    location = f'{lat},{lng}'  # Format the location as a string
    venues = get_all_venues(location, radius=500, venue_type='cafe')
    cafe_list.extend(venues)

    # Respect API rate limits
    time.sleep(3)  # Adjust the delay as per the API's rate limit

In [61]:
places_cafes_df = pd.DataFrame(cafe_list)

# export raw response df to csv
places_cafes_df.to_csv('data/places_cafes_response.csv', index=False)

# drop duplicates
places_cafes_df.drop_duplicates(subset='place_id', inplace=True)

# drop columns
places_cafes_df = places_cafes_df[['name', 'geometry', 'rating', 'user_ratings_total', 'types', 'vicinity', 'place_id']]

# function to create point geometry
def create_point(row):
    # Access the latitude and longitude inside the 'geometry' column
    lat = row['geometry']['location']['lat']
    lng = row['geometry']['location']['lng']
    # Return a Point object with longitude and latitude
    return Point(lng, lat)

# Now, apply this function to each row in the DataFrame to create a new column 'point'
places_cafes_df['point'] = places_cafes_df.apply(create_point, axis=1)

# Finally, convert our DataFrame to a GeoDataFrame
places_gdf = gpd.GeoDataFrame(places_cafes_df, geometry='point')

# Set the CRS for the GeoDataFrame to WGS84 (lat/long)
places_gdf.crs = "EPSG:4326"

# drop geometry column (redundant), types column (incompatible with geojson)
places_gdf.drop(columns=['geometry', 'types'], inplace=True)

places_gdf.head()

,name,rating,user_ratings_total,vicinity,place_id,point
0,Trees Organic Coffee,4.3,1606.0,"450 Granville Street, Vancouver",ChIJx19lhHhxhlQR00Wyx2MBMZc,POINT (-123.11433 49.28484)
1,Tim Hortons,3.5,474.0,"555 West Hastings Street Unit 6, Vancouver",ChIJfe2sYHhxhlQR6YR9clFor1Y,POINT (-123.11199 49.28495)
2,Mink Chocolates Cafe,4.3,1149.0,"863 West Hastings Street, Vancouver",ChIJPxF7YIJxhlQR2GV7dekt2xM,POINT (-123.11539 49.28665)
3,Revolver,4.7,2126.0,"325 Cambie Street, Vancouver",ChIJk5IDu3lxhlQRrQ0XKyJs_mk,POINT (-123.10948 49.28320)
4,Waves Coffee House - Hastings,4.1,424.0,"492 West Hastings Street, Vancouver",ChIJg8oU-HhxhlQRIZKCSINvdrg,POINT (-123.11194 49.28385)


In [62]:
places_gdf.shape

(335, 6)

In [64]:
# export as geojson
places_gdf.to_file('data/places_cafes.geojson', driver='GeoJSON')

### add cafe details (price level, opening_hours)

In [7]:
places_gdf = gpd.read_file('data/geojson/point/places_cafes.geojson')

def get_place_details(place_id, api_key):
    # Define the endpoint URL and parameters
    endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        'place_id': place_id,
        'fields': 'price_level,opening_hours',
        'key': api_key
    }
    
    # Make the request and process the response
    response = requests.get(endpoint_url, params=params)
    result = response.json()
    
    # Extract the price level and periods data
    price_level = result.get('result', {}).get('price_level', None)
    periods = result.get('result', {}).get('opening_hours', {}).get('periods', [])

    return price_level, periods

# Define the process_periods function to calculate open hours based on periods
def process_periods(periods):
    # Initialize a dictionary to store open hours for each day
    open_hours = {day: {"open": None, "close": None} for day in range(7)}
    
    # Process each period
    for period in periods:
        open_day = period['open']['day']
        close_day = period.get('close', {}).get('day')
        open_time = period['open']['time']
        close_time = period.get('close', {}).get('time')
        
        # Update the open_hours dictionary
        open_hours[open_day]['open'] = open_time
        if close_day is not None:
            open_hours[close_day]['close'] = close_time
    
    # Convert open_hours to a format suitable for your analysis
    # e.g., total weekly hours, average daily open hours, etc.
    
    return json.dumps(open_hours)

# Update your DataFrame creation loop
for index, row in places_gdf.iterrows():
    place_id = row['place_id']
    price_level, periods = get_place_details(place_id, api_key)

    # Update the DataFrame with the new information
    places_gdf.loc[index, 'price_level'] = price_level
    # Process periods data to calculate open hours
    places_gdf.loc[index, 'open_hours_info'] = process_periods(periods)

    time.sleep(0.1)  # Be mindful of the API's rate limit

places_gdf.head()

,name,rating,user_ratings_total,vicinity,place_id,geometry,price_level,open_hours_info
0,Trees Organic Coffee,4.3,1606.0,"450 Granville Street, Vancouver",ChIJx19lhHhxhlQR00Wyx2MBMZc,POINT (-123.11433 49.28484),2.0,"{""0"": {""open"": ""0800"", ""close"": ""2200""}, ""1"": ..."
1,Tim Hortons,3.5,474.0,"555 West Hastings Street Unit 6, Vancouver",ChIJfe2sYHhxhlQR6YR9clFor1Y,POINT (-123.11199 49.28495),1.0,"{""0"": {""open"": ""0700"", ""close"": ""2100""}, ""1"": ..."
2,Mink Chocolates Cafe,4.3,1149.0,"863 West Hastings Street, Vancouver",ChIJPxF7YIJxhlQR2GV7dekt2xM,POINT (-123.11539 49.28665),2.0,"{""0"": {""open"": ""0900"", ""close"": ""1700""}, ""1"": ..."
3,Revolver,4.7,2126.0,"325 Cambie Street, Vancouver",ChIJk5IDu3lxhlQRrQ0XKyJs_mk,POINT (-123.10948 49.28320),2.0,"{""0"": {""open"": null, ""close"": null}, ""1"": {""op..."
4,Waves Coffee House - Hastings,4.1,424.0,"492 West Hastings Street, Vancouver",ChIJg8oU-HhxhlQRIZKCSINvdrg,POINT (-123.11194 49.28385),2.0,"{""0"": {""open"": ""0800"", ""close"": ""1800""}, ""1"": ..."


In [8]:
places_gdf.to_file('tmp_cafes_details_response.geojson', driver='GeoJSON')

In [41]:
places_gdf = gpd.read_file('tmp_cafes_details_response.geojson')

In [49]:
places_gdf = gpd.read_file('tmp_cafes_details_response.geojson')

def deserialize_open_hours(row):
    if isinstance(row, dict):
        return row  # Return the dict as is, since it's already deserialized
    return json.loads(row) if row else {}

# Note that you also need to have the convert_time function defined that can handle '0000' as 24 hours:
def convert_time(time_str):
    if time_str == '0000':
        return 24
    hour = int(time_str[:2])
    minute = int(time_str[2:]) / 60
    return hour + minute

def calculate_open_hours(open_hours_info):
    # Initialize variables
    sum_open_weekday, sum_close_weekday, sum_open_weekend, sum_close_weekend = (0, 0, 0, 0)
    weekday_count, weekend_count = (0, 0)
    total_open_hours = 0
    is_24_hour = False

    # Check for 24-hour open case
    if open_hours_info.get('0', {}).get('open') == '0000' and all(open_hours_info.get(str(day), {}).get('open') is None for day in range(1, 7)):
        is_24_hour = True
        total_open_hours = 24 * 7

    if not is_24_hour:
        for key, value in open_hours_info.items():
            # Skip if data is missing
            if not value.get('open') or not value.get('close'):
                continue
            
            # Convert times to decimal hours
            open_time = convert_time(value['open'])
            close_time = convert_time(value['close'])
            if close_time == 0:  # Handle closing time at midnight
                close_time = 24

            # Calculate total open hours
            daily_open_hours = close_time - open_time
            total_open_hours += daily_open_hours

            # Aggregate weekday and weekend times
            if key in ['0', '1', '2', '3', '4']:  # Weekdays
                sum_open_weekday += open_time
                sum_close_weekday += close_time
                weekday_count += 1
            elif key in ['5', '6']:  # Weekend
                sum_open_weekend += open_time
                sum_close_weekend += close_time
                weekend_count += 1

    # Calculate averages
    avg_weekday_open = sum_open_weekday / weekday_count if weekday_count else None
    avg_weekday_close = sum_close_weekday / weekday_count if weekday_count else None
    avg_weekend_open = sum_open_weekend / weekend_count if weekend_count else None
    avg_weekend_close = sum_close_weekend / weekend_count if weekend_count else None

    # Return the calculated values
    return avg_weekday_open, avg_weekday_close, avg_weekend_open, avg_weekend_close, total_open_hours

places_gdf['open_hours_dict'] = places_gdf['open_hours_info'].apply(deserialize_open_hours)

places_gdf.drop(columns='open_hours_info', inplace=True)

for index, row in places_gdf.iterrows():
    # Extract the open_hours_info for the current row
    open_hours_dict = row['open_hours_dict']
    
    # Calculate the open hours using the function
    avg_weekday_open, avg_weekday_close, avg_weekend_open, avg_weekend_close, total_open_hours = calculate_open_hours(open_hours_dict)
    
    # Update the DataFrame with the calculated values
    places_gdf.loc[index, 'avg_weekday_open'] = avg_weekday_open
    places_gdf.loc[index, 'avg_weekday_close'] = avg_weekday_close
    places_gdf.loc[index, 'avg_weekend_open'] = avg_weekend_open
    places_gdf.loc[index, 'avg_weekend_close'] = avg_weekend_close
    places_gdf.loc[index, 'total_open_hours'] = total_open_hours

# replace 0 with NaN for total_open_hours
places_gdf['total_open_hours'] = places_gdf['total_open_hours'].replace(0, np.nan)

# Define the function to categorize opening and closing times
def categorize_time(time, early_cutoff, late_cutoff, total_open_hours):
    if total_open_hours == 168:
        return '24/7'
    elif time <= early_cutoff:
        return 'Early'
    elif time >= late_cutoff:
        return 'Late'
    else:
        return 'Standard'

# Apply the categorization function to each relevant column
for time_type in ['weekday_open', 'weekday_close', 'weekend_open', 'weekend_close']:
    # Determine the cutoffs for each time type
    mean_time = places_gdf['avg_' + time_type].mean()
    std_time = places_gdf['avg_' + time_type].std()
    early_cutoff = round(mean_time - std_time, 0)
    late_cutoff = round(mean_time + std_time, 0)

    # Apply categorization considering total_open_hours
    places_gdf[time_type + '_category'] = places_gdf.apply(
        lambda row: categorize_time(row['avg_' + time_type], early_cutoff, late_cutoff, row['total_open_hours']), axis=1)

# Function to replace category values based on the time type
def update_category(value, col_name):
    if value == '24/7':
        return 'Open 24/7' if 'open' in col_name else 'Open 24/7'
    if value == 'Early':
        return 'Early Open (< 6AM)' if 'open' in col_name else 'Early Close (< 3PM)'
    elif value == 'Late':
        return 'Late Open (> 11AM)' if 'open' in col_name else 'Late Close (> 10PM)'
    else:  # 'Standard'
        return 'Standard Open' if 'open' in col_name else 'Standard Close'

# Columns to update
time_categories = ['weekday_open_category', 'weekday_close_category', 
                   'weekend_open_category', 'weekend_close_category']

# Update the DataFrame
for col in time_categories:
    places_gdf[col] = places_gdf[col].apply(lambda x: update_category(x, col))

places_gdf.drop(columns=['open_hours_dict', 'avg_weekday_open', 'avg_weekday_close', 'avg_weekend_open', 'avg_weekend_close'], inplace=True)

places_gdf.rename(columns={'weekday_open_category': 'open_weekday', 'weekday_close_category': 'close_weekday', 'weekend_open_category': 'open_weekend', 'weekend_close_category': 'close_weekend', 'total_open_hours': 'weekly_open_hours'}, inplace=True)

places_gdf.head()

,name,rating,user_ratings_total,vicinity,place_id,price_level,geometry,weekly_open_hours,open_weekday,close_weekday,open_weekend,close_weekend
0,Trees Organic Coffee,4.3,1606.0,"450 Granville Street, Vancouver",ChIJx19lhHhxhlQR00Wyx2MBMZc,2.0,POINT (-123.11433 49.28484),103.0,Standard Open,Late Close (> 10PM),Standard Open,Late Close (> 10PM)
1,Tim Hortons,3.5,474.0,"555 West Hastings Street Unit 6, Vancouver",ChIJfe2sYHhxhlQR6YR9clFor1Y,1.0,POINT (-123.11199 49.28495),106.5,Early Open (< 6AM),Late Close (> 10PM),Early Open (< 6AM),Standard Close
2,Mink Chocolates Cafe,4.3,1149.0,"863 West Hastings Street, Vancouver",ChIJPxF7YIJxhlQR2GV7dekt2xM,2.0,POINT (-123.11539 49.28665),63.5,Standard Open,Standard Close,Standard Open,Standard Close
3,Revolver,4.7,2126.0,"325 Cambie Street, Vancouver",ChIJk5IDu3lxhlQRrQ0XKyJs_mk,2.0,POINT (-123.10948 49.28320),57.0,Standard Open,Standard Close,Standard Open,Standard Close
4,Waves Coffee House - Hastings,4.1,424.0,"492 West Hastings Street, Vancouver",ChIJg8oU-HhxhlQRIZKCSINvdrg,2.0,POINT (-123.11194 49.28385),75.0,Standard Open,Standard Close,Standard Open,Standard Close


In [50]:
# export as geojson
places_gdf.to_file('data/geojson/point/cafes_details.geojson', driver='GeoJSON')

In [10]:
places_gdf['hours_category'] = None

places_gdf.head()

,name,rating,user_ratings_total,vicinity,place_id,geometry,price_level,open_hours_info,hours_category
0,Trees Organic Coffee,4.3,1606.0,"450 Granville Street, Vancouver",ChIJx19lhHhxhlQR00Wyx2MBMZc,POINT (-123.11433 49.28484),2.0,"{""0"": {""open"": ""0800"", ""close"": ""2200""}, ""1"": ...",None
1,Tim Hortons,3.5,474.0,"555 West Hastings Street Unit 6, Vancouver",ChIJfe2sYHhxhlQR6YR9clFor1Y,POINT (-123.11199 49.28495),1.0,"{""0"": {""open"": ""0700"", ""close"": ""2100""}, ""1"": ...",None
2,Mink Chocolates Cafe,4.3,1149.0,"863 West Hastings Street, Vancouver",ChIJPxF7YIJxhlQR2GV7dekt2xM,POINT (-123.11539 49.28665),2.0,"{""0"": {""open"": ""0900"", ""close"": ""1700""}, ""1"": ...",None
3,Revolver,4.7,2126.0,"325 Cambie Street, Vancouver",ChIJk5IDu3lxhlQRrQ0XKyJs_mk,POINT (-123.10948 49.28320),2.0,"{""0"": {""open"": null, ""close"": null}, ""1"": {""op...",None
4,Waves Coffee House - Hastings,4.1,424.0,"492 West Hastings Street, Vancouver",ChIJg8oU-HhxhlQRIZKCSINvdrg,POINT (-123.11194 49.28385),2.0,"{""0"": {""open"": ""0800"", ""close"": ""1800""}, ""1"": ...",None


In [38]:
test_df = places_gdf[:5]

test_df['open_hours_info']

0    {"0": {"open": "0800", "close": "2200"}, "1": ...
1    {"0": {"open": "0700", "close": "2100"}, "1": ...
2    {"0": {"open": "0900", "close": "1700"}, "1": ...
3    {"0": {"open": null, "close": null}, "1": {"op...
4    {"0": {"open": "0800", "close": "1800"}, "1": ...
Name: open_hours_info, dtype: object

In [39]:
def deserialize_open_hours(row):
    return json.loads(row) if row else {}

test_df.loc[:, 'open_hours_info'] = test_df['open_hours_info'].apply(deserialize_open_hours)

test_df['open_hours_info']


0    {'0': {'open': '0800', 'close': '2200'}, '1': ...
1    {'0': {'open': '0700', 'close': '2100'}, '1': ...
2    {'0': {'open': '0900', 'close': '1700'}, '1': ...
3    {'0': {'open': None, 'close': None}, '1': {'op...
4    {'0': {'open': '0800', 'close': '1800'}, '1': ...
Name: open_hours_info, dtype: object

In [40]:
test_df['open_hours_info'][0]['0']

{'open': '0800', 'close': '2200'}

In [ ]:
test_df['hours_category'] = None

def categorize_hours(row):
    

In [21]:
for hours in test_df:
    hours = json.loads(hours)

test_df[0]

'{'

In [ ]:
def deserialize_open_hours(row):
    return json.loads(row) if row else {}

# Note that you also need to have the convert_time function defined that can handle '0000' as 24 hours:
def convert_time(time_str):
    if time_str == '0000':
        return 24
    hour = int(time_str[:2])
    minute = int(time_str[2:]) / 60
    return hour + minute

def calculate_open_hours(open_hours_dict):
    # Initialize variables
    sum_open_weekday, sum_close_weekday, sum_open_weekend, sum_close_weekend = (0, 0, 0, 0)
    weekday_count, weekend_count = (0, 0)
    total_open_hours = 0
    is_24_hour = False

    # Check for 24-hour open case
    if open_hours_dict.get('0', {}).get('open') == '0000' and all(open_hours_dict.get(str(day), {}).get('open') is None for day in range(1, 7)):
        is_24_hour = True
        total_open_hours = 24 * 7

    if not is_24_hour:
        for key, value in open_hours_dict.items():
            # Skip if data is missing
            if not value.get('open') or not value.get('close'):
                continue
            
            # Convert times to decimal hours
            open_time = convert_time(value['open'])
            close_time = convert_time(value['close'])
            if close_time == 0:  # Handle closing time at midnight
                close_time = 24

            # Calculate total open hours
            daily_open_hours = close_time - open_time
            total_open_hours += daily_open_hours

            # Aggregate weekday and weekend times
            if key in ['0', '1', '2', '3', '4']:  # Weekdays
                sum_open_weekday += open_time
                sum_close_weekday += close_time
                weekday_count += 1
            elif key in ['5', '6']:  # Weekend
                sum_open_weekend += open_time
                sum_close_weekend += close_time
                weekend_count += 1

    # Calculate averages
    avg_weekday_open = sum_open_weekday / weekday_count if weekday_count else None
    avg_weekday_close = sum_close_weekday / weekday_count if weekday_count else None
    avg_weekend_open = sum_open_weekend / weekend_count if weekend_count else None
    avg_weekend_close = sum_close_weekend / weekend_count if weekend_count else None

    # Return the calculated values
    return avg_weekday_open, avg_weekday_close, avg_weekend_open, avg_weekend_close, total_open_hours